# Test the KSWIN function to detect drift in stream

This is to test if KSWIN is really good in drift detection, if yes we can apply it on IForest ASD to detect drift in windows in order to update the moel if neccessary

## Install skmultiflow if needed
You need to install git

In [62]:
#print("scikit-multiflow package installation")
#!pip install -U git+https://github.com/scikit-multiflow/scikit-multiflow

In [63]:
try:
    import skmultiflow
except ImportError as e:
    print("scikit-multiflow package installation")
    !pip install -U git+https://github.com/scikit-multiflow/scikit-multiflow

## Importations and configurations

In [64]:
%matplotlib notebook
import matplotlib as plt
plt.interactive(True)
from source import functions
func = functions.Comparison()
import datetime
#from skmultiflow.data import AnomalySineGenerator

## General parameters for the evaluation

## Execution Function
https://github.com/scikit-multiflow/scikit-multiflow/blob/master/src/skmultiflow/drift_detection/kswin.py

In [68]:
#************************ Execution settings *******************************

window = 100 # Number of sample in one window, window size.
max_sample = 10000 # Total size of data to examine (windows number = max_sample/window)
window_number = int(max_sample/window) # The number of window to save on .csv file


In [108]:
def execute_comparision(stream, window_size=100, window_number=1000):
    from skmultiflow.drift_detection import KSWIN
    import pandas as pd
    import numpy as np
    # Initialize KSWIN and a data stream
    first_window = stream.next_sample(window_size)[0][:,0]
    #print(first_window)
    kswin = KSWIN(alpha=0.01, data=first_window)
    # Store detections
    detections = []
    # Process stream via KSWIN and print detections
    for i in range(window_number-1):
        #data = stream.next_sample(window_size)
        data = stream.next_sample(window_size)
        #print(data)
        #print(data[0][:,0])
        batch = data[0][0][0]
        #X = pd.DataFrame(data)
        #batch = np.ndarray(X)
        #batch = data[0][0][0]
        #print(batch)
        #n_features = data[0].shape[1]
        #print("n_features == "+str(n_features))
        #if n_features==4:
        #    batch = data[0][0][0][0]
        #elif n_features == 2:
        #    batch = data[0][0]
        #elif n_features == 1:
        #    batch = data[0]
        #else: #It will work for n_features == 3
        #    batch = data[0][0][0]
        #print(batch)
        kswin.add_element(batch)
        if kswin.detected_change():
            print("\rIteration {}".format(i))
            #print("\r KSWINReject Null Hyptheses")
            detections.append(i)
    print("Number of detections: "+str(len(detections)))

## Applied On Simple Stream Dataset


In [109]:
dataset_name = "Generator"
test_name = dataset_name+'_'+str(datetime.datetime.now())

stream = func.get_dataset(dataset_name=dataset_name, classification_function=0,noise_percentage=0.0, random_state=1)

execute_comparision(stream=stream, window_size=window, window_number=window_number)

Iteration 42
Number of detections: 1


## Applied On Stream data containning anomalies

In [110]:
dataset_name = "AnomalySineGenerator"
test_name = dataset_name+'_'+str(datetime.datetime.now())

anomalies_percentage = 0.1
n_anomalies= int(max_sample*anomalies_percentage)
#print(str(n_anomalies))
stream = func.get_anomalies_data_generated(n_samples=max_sample, n_anomalies=n_anomalies, contextual=False,
                 n_contextual=0.0, shift=4, noise=0.0, replace=True, random_state=None)

execute_comparision(stream=stream, window_size=window, window_number=window_number)

Iteration 16
Iteration 92
Number of detections: 2


## Applied On Simple ConceptDrift Stream Dataset

In [111]:
dataset_name = "DriftStreamGenerator"
test_name = dataset_name+'_'+str(datetime.datetime.now())
#drift_rate = 0.7
stream = func.get_dataset(dataset_name=dataset_name, classification_function=0, noise_percentage=0.0, random_state=1,
                         drift_classification_function = 3, drift_random_state = 112, drift_noise_percentage = 0.0,
                          drift_start_position = 110, drift_width = 50, n_num_features = 2, n_cat_features = 0)

execute_comparision(stream=stream, window_size=window, window_number=window_number)

Number of detections: 0
